### REVISAR SI INCLUIR TODAS ESAS LIBRERIAS

# METER CAT PRICE Y HACER GRAFICOS

### TOP 50 References

In this section, I am going to analyze which are the references most sold in the last 6 months (from 1 october 2018 to 31 march 2019).

First of all, we set as path the directory which contains the data

In [52]:
import os

path = "/home/miguel/Escritorio/TFM"
os.chdir(path)

In [53]:
pwd

'/home/miguel/Escritorio/TFM'

Secondly, I import the libraries that I am going to work in this script

In [54]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

After that, I load xlsx file with the required data:
    - References
    - Name References
    - Date time 
    - Net incomes
    - Units sold per transaction

In [55]:
df_bw = pd.read_excel("BW.xlsx", sheet_name="BW")

In [56]:
df_bw.head(5)

,Referencia,Name,T Día natural,Valor neto pedidos,Cantidad en unidades (pedidos)
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",11.09.2017,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",21.09.2017,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",08.10.2017,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",16.10.2017,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",21.10.2017,16.08,1


I also check the shape and info of this data

In [57]:
df_bw.shape

(37614, 5)

In [58]:
df_bw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37614 entries, 0 to 37613
Data columns (total 5 columns):
Referencia                        37614 non-null int64
Name                              37614 non-null object
T Día natural                     37614 non-null object
Valor neto pedidos                37614 non-null float64
Cantidad en unidades (pedidos)    37614 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 1.4+ MB


It is all right.

Then, I continue renaming the columns with the purpose of removing black spaces

In [59]:
df_bw = df_bw.rename(index=str, columns={"Referencia":"Reference", "T Día natural":"Date", "Valor neto pedidos":"Net_Incomes", "Cantidad en unidades (pedidos)":"Units_sold"})

In [60]:
df_bw.head(5)

,Reference,Name,Date,Net_Incomes,Units_sold
0,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",11.09.2017,18.93,1
1,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",21.09.2017,18.93,1
2,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",08.10.2017,16.08,1
3,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",16.10.2017,16.08,1
4,4623,"ESPEJO PUERTA ""LCC"" BLANCO POLIETILENO",21.10.2017,16.08,1


Now, I filter the transactions from 1 october 2018 to 31 march 2019.

To develop correctly this filter, it is necessary to change "Date" column from object type to datetime

In [61]:
from datetime import datetime

In [62]:
df_bw["Date"] = pd.to_datetime(df_bw["Date"], format = "%d.%m.%Y")

-----------------
# HASTA AQUI SE PUEDE HACER UNA FUNCION
-----------------

In [63]:
df_bw_filter = df_bw[df_bw["Date"]>"10.01.2018"]

In [64]:
df_bw_filter.head(5)

,Reference,Name,Date,Net_Incomes,Units_sold
33,6558,JARRÓN CILÍNDRICO VIDRIO,2018-12-09,13.97,1
37,6565,JARRÓN ESFERA VIDRIO,2018-11-08,11.08,1
38,6565,JARRÓN ESFERA VIDRIO,2018-11-23,22.14,2
39,6565,JARRÓN ESFERA VIDRIO,2019-02-04,14.90,1
40,6565,JARRÓN ESFERA VIDRIO,2019-03-12,12.31,1


In [65]:
df_bw_filter.shape

(15032, 5)

Once I have obtained the selected data, I group by Reference and I enumerate how many days each reference has been sold during that period in order to be able to find the most sold products with high frequency.

In [66]:
df_bw_groupby = df_bw.groupby(by="Reference").agg({"Date":"count", "Units_sold":"sum","Net_Incomes":"sum"}).sort_values(by='Units_sold',ascending=False)
df_bw_groupby.head(5)

,Date,Units_sold,Net_Incomes
Reference,,,
265400,82,1092,0.0
265600,82,1092,0.0
251900,82,904,0.0
10040,58,562,0.0
10048,48,506,0.0


In [67]:
df_bw_groupby.describe()

,Date,Units_sold,Net_Incomes
count,8536.000000,8536.000000,8536.000000
mean,4.406514,6.887067,258.291885
std,7.910064,26.817064,697.498123
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,32.230000
50%,2.000000,3.000000,77.320000
75%,4.000000,6.000000,217.440000
max,172.000000,1092.000000,14050.750000


After grouping and checking the new dataframe, I should select the most suitable references for this study following these steps:
   - Net_Incomes > 0€ in BW
   - At least more than 30 days with a transaction
   - At least more than 30 days of exposition in Adwords
   - Cost > 0€ in Adwords

Firstly, I filter net incomes over a 0€ and a minimum of 30 days with a transaction

In [68]:
df_bw_groupby = df_bw_groupby[(df_bw_groupby["Net_Incomes"]!=0) & (df_bw_groupby["Date"]>=30)]

Second, I must load the file which contains Adwords data:

In [100]:
df_ads_2018_19 = pd.read_csv("TFM_Datos_2018-19.csv", sep=";")

/home/miguel/my_project_python/my_project_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [101]:
df_ads_2018_19.head(5)

,ID de producto,Campaña,ID de la campaña,Tipo de producto (primer nivel),Tipo de producto (segundo nivel),Día,Etiqueta personalizada 1,CPC máximo predeterminado del grupo de anuncios,Moneda,Impresiones,...,CTR,CPC medio,Coste,Conversiones,Todas las conversiones,Tasa de conversión,Coste/conv.,Coste/todas las conversiones,Valor conv./coste,Valor de todas las conversiones/coste
0,127156,Shop_Smart_N1_Téxtil_Hogar_Y19_W5,1695676698,textil hogar,alfombras de vinilo,20/02/2019,200,"0,01",EUR,153,...,"0,00 %",0,0,0,0,"0,00 %",0,0,0,0
1,104268,Shop_Decoración_N1_Y18_W37,1559890739,decoración para tu casa,figuras decorativas,16/10/2018,40,"0,15",EUR,9,...,"0,00 %",0,0,0,0,"0,00 %",0,0,0,0
2,123872,Shop_Smart_N1_Téxtil_Hogar_Y19_W5,1695676698,textil hogar,edredones,25/03/2019,80,"0,01",EUR,1,...,"0,00 %",0,0,0,0,"0,00 %",0,0,0,0
3,110275,Shop_Navidad_Bolas de navidad_Y18_W12,1323358035,decoración de navidad,bolas de navidad,18/12/2018,10,"0,2",EUR,7,...,"0,00 %",0,0,0,0,"0,00 %",0,0,0,0
4,49843,Shop_Smart_N1_Baño_Y19_W5,1623532023,baño,"dispensadores de baño, jaboneras y portacepillos",16/02/2019,20,"0,01",EUR,21,...,"0,00 %",0,0,0,0,"0,00 %",0,0,0,0


In [102]:
df_ads_2018_19.shape

(447539, 21)

In [103]:
df_ads_2018_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447539 entries, 0 to 447538
Data columns (total 21 columns):
ID de producto                                     447539 non-null int64
Campaña                                            447539 non-null object
ID de la campaña                                   447539 non-null int64
Tipo de producto (primer nivel)                    447539 non-null object
Tipo de producto (segundo nivel)                   447539 non-null object
Día                                                447539 non-null object
Etiqueta personalizada 1                           447539 non-null object
CPC máximo predeterminado del grupo de anuncios    447539 non-null object
Moneda                                             447539 non-null object
Impresiones                                        447539 non-null int64
Clics                                              447539 non-null int64
CTR                                                447539 non-null object
CPC m

After checking the file, let choosing the required columns to analyze cost and day' transactions

In [104]:
columns_interest = ["ID de producto", "Día", "Coste", "Impresiones"]
df_ads = df_ads_2018_19[columns_interest]

In [105]:
df_ads.head(5)

,ID de producto,Día,Coste,Impresiones
0,127156,20/02/2019,0,153
1,104268,16/10/2018,0,9
2,123872,25/03/2019,0,1
3,110275,18/12/2018,0,7
4,49843,16/02/2019,0,21


Once, I have filtered, I should repeat the same process as in the previous file:
    - Rename columns
    - Column "Día" from string to datetime and columns "Coste" and "Impresiones" from string to float64
    - Groupby by "ID de producto" and agreggate "Día" as count and "Coste" as sum(it is not necessary to filter by date because the file contains the period analyzed)

In [106]:
#Step1: Rename
df_ads = df_ads.rename(index=str, columns={"ID de producto":"Reference", "Día":"Date", "Coste":"Cost", "Impresiones":"Impresions"})

#Step2: from string to datetime and float
df_ads["Date"] = pd.to_datetime(df_ads["Date"], format = "%d/%m/%Y")
df_ads["Cost"] = df_ads["Cost"].str.replace(",",".").astype(float)

#Step3: grouping
df_ads_groupby = df_ads.groupby(by="Reference").agg({"Date":"count", "Cost":"sum", "Impresions":"sum"}).sort_values(by='Cost',ascending=False)

In [107]:
df_ads["Cost"].dtype

dtype('float64')

In [108]:
df_ads["Impresions"].dtype

dtype('int64')

In [109]:
df_ads_groupby.head(5)

,Date,Cost,Impresions
Reference,,,
64534,182,2701.53,1165107
123838,132,1803.89,398687
98064,112,1427.50,482281
103808,174,1370.33,521766
64531,185,1365.16,562220


After this last step, I filter by: "Cost">0 and "Date">30

In [110]:
df_ads_groupby = df_ads_groupby[(df_ads_groupby["Cost"]!=0) & (df_ads_groupby["Date"]>=30)]

In [111]:
df_ads_groupby.tail(5)

,Date,Cost,Impresions
Reference,,,
88656,42,0.01,107
107553,126,0.01,365
122605,48,0.01,243
110540,39,0.01,558
48234,83,0.01,304


OK!



Let continue merging the dataframes "df_ads_groupby" and "df_bw_groupby" by Reference in order to select the top 50 products to analyze in this study.

In [112]:
df_bw_groupby.head(5)

,Date,Units_sold,Net_Incomes
Reference,,,
8141600,55,416,8533.68
82462,38,363,11208.61
82035,82,336,8336.26
61119,172,263,11613.23
95203,46,182,2097.29


In [113]:
df_ads_groupby.head(5)

,Date,Cost,Impresions
Reference,,,
64534,182,2701.53,1165107
123838,132,1803.89,398687
98064,112,1427.50,482281
103808,174,1370.33,521766
64531,185,1365.16,562220


In [114]:
merge_bw_ads = df_bw_groupby.merge(df_ads_groupby,
           how='inner',
           left_on=['Reference'],
           right_on=['Reference'])

I sort values by "Units_Sold" and I select the top 50:

In [115]:
merge_bw_ads.sort_values(by=["Units_sold"], ascending=False).head(50)

,Date_x,Units_sold,Net_Incomes,Date_y,Cost,Impresions
Reference,,,,,,
8141600,55,416,8533.68,126,379.78,174498
82462,38,363,11208.61,56,121.41,57854
82466,41,180,6270.32,126,158.68,96652
8146000,98,179,4401.90,39,263.21,106009
122539,99,148,5667.47,110,965.58,282555
123838,87,143,2636.63,132,1803.89,398687
61096,104,142,5365.26,180,481.68,128744
89051,115,142,6736.89,149,183.95,67112
61731,91,126,10227.65,63,855.59,262189


I will work this study with this group of references.

However, before saving this dataframe as csv, I am going to rename columns


In [118]:
top_50_references = merge_bw_ads.sort_values(by=["Units_sold"], ascending=False).head(50)

In [119]:
top_50_references = top_50_references.rename(index=str, columns={"Date_x":"Date_BW", "Date_y":"Date_ADS"})

In [120]:
top_50_references.head(5)

,Date_BW,Units_sold,Net_Incomes,Date_ADS,Cost,Impresions
Reference,,,,,,
8141600,55,416,8533.68,126,379.78,174498
82462,38,363,11208.61,56,121.41,57854
82466,41,180,6270.32,126,158.68,96652
8146000,98,179,4401.90,39,263.21,106009
122539,99,148,5667.47,110,965.58,282555


Finally, after renaming the columns, I save the data in a csv:

In [121]:
path = "/home/miguel/my_project_python/my_project_env/tfm/datasets"
os.chdir(path)

In [122]:
top_50_references.to_csv("top_50_references.csv", sep="\t")